In [69]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
data = pd.read_csv('alphanet_codetest_daily.csv')
# 定义数据图片开始和结束的时间
time_start = 20200923
time_end = 20210923


stock = pd.DataFrame(data.groupby('ticker')['timestamp'].count() == 487).reset_index()
merge = data.merge(stock, on='ticker', how='left')
merge = merge[merge['timestamp_y'] == True]
merge['timestamp'] = merge['timestamp_x']
merge = merge[['timestamp','ticker','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover']] # feature
merge = merge[(merge['timestamp'] >= time_start) & (merge['timestamp'] <= time_end)]
merge

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover
929205,20200923,1,15.59,15.83,15.51,15.63,15.6836,599200.47,0.3854,0.3088,0.6967
929206,20200923,2,29.10,29.20,28.55,28.70,28.7589,386288.56,-0.8978,0.3976,0.5969
929207,20200923,4,23.75,23.82,23.36,23.69,23.5851,17510.01,0.2964,2.0865,2.9163
929208,20200923,5,2.72,2.74,2.69,2.70,2.7085,82201.17,-0.7353,0.7770,0.9954
929209,20200923,6,6.20,6.64,6.12,6.38,6.3930,447987.98,3.5714,3.3226,5.1952
...,...,...,...,...,...,...,...,...,...,...,...
1958151,20210923,688168,52.80,53.30,52.00,52.45,52.5581,5919.73,1.0792,2.0771,2.1589
1958162,20210923,688188,446.80,449.00,425.20,438.14,438.5992,4444.64,-1.6830,1.6540,1.6540
1958216,20210923,688321,39.20,39.30,38.64,38.90,38.8391,14252.01,0.0772,0.5631,0.6717
1958221,20210923,688333,199.87,201.01,192.01,200.34,196.4810,4126.57,0.6177,0.9092,0.9092


In [70]:
day_back = 3 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天


x , y , x_delay = [] , [], [] # 初始数据集

for count , ticker in enumerate(merge['ticker'].drop_duplicates()):
#     print(count)
    one_data = merge[merge['ticker'] == ticker]
    one_data['pct_change_shift'] = (one_data['close'].shift(-day_back) - one_data['close']) / one_data['close']
    one_data = one_data.set_index(['timestamp','ticker'])
    one_data = one_data.dropna() # 丢弃因为回溯而产生的空值
    array = np.array(one_data)

    for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        x.append(array[i:i+day,:-1].T)
        y.append(array[i+day-1][-1])
x  , y = np.array(x) , np.array(y).reshape(-1,1) # x = (153, 9, 30) , y = (153,1)
x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2]) # x = (153, 1, 9, 30)
print("x.shape: ",x.shape)
print("y.shape: ",y.shape)

C:\Users\Xinfei\AppData\Local\Temp/ipykernel_3256/373581283.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_data['pct_change_shift'] = (one_data['close'].shift(-day_back) - one_data['close']) / one_data['close']


x.shape:  (254800, 1, 9, 30)
y.shape:  (254800, 1)


In [72]:
class Convolutional(object):
    def __init__(self,data,stride):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = np.array(data)
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2] # 9
        self.num , self.num_rev = self.generate_Num_and_ReversedNum(self.feat_num)
        self.conv_feat = len(self.num)
        self.step_list = self.generate_Step_List(self.data_length,self.stride)
        self.extracted_data = self.Extraction(self.data,self.feat_num,self.conv_feat,self.stride)
    def Extraction(self,data,feat_num,conv_feat,stride):
        print("------Start Extraction------")
        batch = nn.BatchNorm1d(conv_feat,affine=True)
        batch2 = nn.BatchNorm1d(feat_num,affine=True)
        conv1 = self.ts_cov4d(self.data,self.stride,self.num,self.num_rev,self.step_list)
        conv2 = self.ts_corr4d(self.data,self.stride,self.num,self.num_rev,self.step_list,conv1)
        conv2 = torch.tanh(conv2)
        bc1 = batch(conv1.to(torch.float))
        bc2 = batch(conv2.to(torch.float))
        conv3 = self.ts_stddev4d(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc3 = batch2(conv3)
        conv4 = self.ts_zscore(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc4 = batch2(conv4)
        conv5 = self.ts_return(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc5 = batch2(conv5)
        conv6 = self.ts_decaylinear(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc6 = batch2(conv6)

        feat_cat = torch.cat([bc1,bc2,bc3,bc4,bc5,bc6],axis = 1) # 特征聚合
        shape = feat_cat.shape
        feat_cat = feat_cat.reshape(shape[0],1,shape[1],shape[2])
        print("Convolutional shape: ", feat_cat.shape)
        return feat_cat
    def generateC(self,l1):
        if len(l1) == 1:
            return []
        v = [[l1[0],i] for i in l1[1:]]
        l1 = l1[1:]
        return v+ self.generateC(l1)
    def generate_Num_and_ReversedNum(self,feat_nums):
        list1 = list(range(feat_nums))
        num = self.generateC(list1)
        num_rev = []
        for l in num:
            l1 = l.copy()
            l1.reverse()
            num_rev.append(l1)
        return num , num_rev
    def generate_Step_List(self,data_length,stride):
        # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
    
    
    """ Main Extraction"""
    def ts_cov4d(self,data,stride,num,num_rev,step_list):
        '''计算4维数据的协方差'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        #计算的过程中务必保持keepdims=True
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
            sub_data2 = data[:,:,num_rev,start:end]
            mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
            mean2 = sub_data2.mean(axis = 4,keepdims = True)
            spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
            spread2 = sub_data2 - mean2
            cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
            l.append(cov)
        corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,self.conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
        final = torch.from_numpy(corr)
        print("------Finished ts_cov4d----output shape: ",final.shape)
        return final
    def ts_corr4d(self,data,stride,num,num_rev,step_list,cov):
        '''计算4维数据的相关系数'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,num,start:end]
            sub_data2 = data[:,:,num_rev,start:end]
            std1 = sub_data1.std(axis = 4,keepdims = True)
            std2 = sub_data2.std(axis = 4,keepdims = True)
            std = (std1*std2).mean(axis = 3,keepdims = True)
            del std1 , std2 # 释放内存
            l.append(std)
        std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,self.conv_feat,len(step_list)-1)
        std[std == 0] = 1e-9
        fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
        final = cov / torch.from_numpy(std) * fct
        del fct , std
        print("------Finished ts_corr4d----output shape: ",final.shape)
        return final
    def ts_stddev4d(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            std1 = sub_data1.std(axis = 3,keepdims = True)
            l.append(std1)
            del std1
        std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        print("------Finished ts_stddev4d----output shape: ",torch.from_numpy(std).shape)
        return torch.from_numpy(std)
    def ts_zscore(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            mean = sub_data1.mean(axis = 3,keepdims = True)
            std = sub_data1.std(axis = 3,keepdims = True)
            std[std == 0] = 1e-9
            z_score = mean/std
            l.append(z_score)
        z_score = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
#         z_score[z_score >= 6] = 6
        print("------Finished ts_zscore----output shape: ",torch.from_numpy(z_score).shape)
        return torch.from_numpy(z_score)
    def ts_return(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        data[data == 0] = 1e-9
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            ret = sub_data1[:,:,:,-1]/sub_data1[:,:,:,0] - 1
            l.append(ret)
        z_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        z_data[z_data > 1] = 1
        print("------Finished ts_return----output shape: ",torch.from_numpy(z_data).shape)
        return torch.from_numpy(z_data)
    def ts_decaylinear(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            time_spread = end - start
            weight = np.arange(1,time_spread+1)
            weight = weight/(weight.sum())
            sub_data1 = (data[:,:,:,start:end]*weight).mean(axis = 3,keepdims = True)
            l.append(sub_data1)
        decay_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        final = torch.from_numpy(decay_data)
        print("------Finished ts_decaylinear----output shape: ",final.shape)
        return final

In [73]:
class Pooling(object):
    def __init__(self,data,stride):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = data.detach().numpy()
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2] # 9
        self.step_list = self.generate_Step_List(self.data_length,self.stride)
        self.extracted_data = self.Extraction(self.data,self.feat_num,self.stride)
    def Extraction(self,data,feat_num,stride):
        print("------Start Pooling------")
        # Pooling
        ts_max = self.ts_pool(feat_cat,self.stride,self.feat_num,self.step_list,method = 'max')
        ts_max = nn.BatchNorm1d(108,affine = True)(ts_max)
        ts_min = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'min')
        ts_min = nn.BatchNorm1d(108,affine = True)(ts_min)
        ts_mean = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'mean')
        ts_mean = nn.BatchNorm1d(108,affine = True)(ts_mean)
        data_pool = torch.cat([ts_max,ts_min,ts_mean],axis = 1)
        data_pool = data_pool.flatten(start_dim = 1)
        print("Pooling shape: ",data_pool.shape )
        return data_pool
    def generate_Step_List(self,data_length,stride):
        # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
    def ts_pool(self,data,stride,feat_num,step_list,method):
        if type(data) == torch.Tensor:
            data = data.detach().numpy()
        if data.shape[-1] <= stride:
            step_list = [0,data.shape[-1]]
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in range(len(step_list)-1):
            start = step_list[i]
            end = step_list[i+1]
            if method == 'max':
                sub_data1 = data[:,:,:,start:end].max(axis = 3,keepdims = True)
            if method == 'min':
                sub_data1 = data[:,:,:,start:end].min(axis = 3,keepdims = True)
            if method == 'mean':
                sub_data1 = data[:,:,:,start:end].mean(axis = 3,keepdims = True)
            l.append(sub_data1)
        try:
            pool_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list) - 1)
        except:
            pool_data = np.squeeze(np.array(l)).reshape(-1,feat_num,len(step_list) - 1)
        return torch.from_numpy(pool_data)

In [74]:
convolutional = Convolutional(x,10)
feat_cat = convolutional.extracted_data
pooling = Pooling(feat_cat,3)
x = pooling.extracted_data.detach().numpy()
print("x.shape : ",x.shape)
print("y.shape : ",y.shape)

------Start Extraction------
------Finished ts_cov4d----output shape:  torch.Size([254800, 36, 3])
------Finished ts_corr4d----output shape:  torch.Size([254800, 36, 3])
------Finished ts_stddev4d----output shape:  torch.Size([254800, 9, 3])
------Finished ts_zscore----output shape:  torch.Size([254800, 9, 3])
------Finished ts_return----output shape:  torch.Size([254800, 9, 3])
------Finished ts_decaylinear----output shape:  torch.Size([254800, 9, 3])
Convolutional shape:  torch.Size([254800, 1, 108, 3])
------Start Pooling------
Pooling shape:  torch.Size([254800, 324])
x.shape :  (254800, 324)
y.shape :  (254800, 1)


In [94]:
import torch
from torch import nn
class AlphaNet(nn.Module):
    def __init__(self,factor_num,fully_connect_layer_neural):
        # super 父类，调用父类的构造，这一步必须有
        # 第一个参数为定义类的名称，第二个为self
        super(AlphaNet,self).__init__()
        self.fc1_neuron = (factor_num * (factor_num -1) + 4 * factor_num)* 3
        self.fc2_neuron = fully_connect_layer_neural
        self.fc1 = torch.nn.Linear(self.fc1_neuron,self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron,1)
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred

In [76]:
total_length = x.shape[0]

trainx , trainy  , testx , testy = x[:int(total_length*0.7)] , y[:int(total_length*0.7)] , x[int(total_length*0.7):] , y[int(total_length*0.7):]
trainx , trainy  , testx , testy = torch.from_numpy(trainx) , torch.from_numpy(trainy) , torch.from_numpy(testx) , torch.from_numpy(testy)
print('trainx size: ', trainx.size())
print('trainy size: ', trainy.size())
print('testx size: ', testx.size())
print('testy size: ', testy.size())

trainx size:  torch.Size([178360, 324])
trainy size:  torch.Size([178360, 1])
testx size:  torch.Size([76440, 324])
testy size:  torch.Size([76440, 1])


In [90]:
import torch.utils.data as Data
train_dataset = Data.TensorDataset(trainx, trainy)
test_dataset = Data.TensorDataset(testx, testy)
batch_size = 128
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

In [95]:
import torch.optim as optim 
from tqdm import tqdm
from torch.autograd import Variable

alphanet = AlphaNet(9,30)
print(alphanet)

LR = 0.000001
loss_function = nn.MSELoss()
optimizer = optim.RMSprop(alphanet.parameters(), lr=LR, alpha=0.9)
epoch_num = 30


for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _,(data, label) in enumerate(train_loader):
        data = Variable(data).float()
        pred = alphanet(data)
        label = Variable(label).float()
#         label = label.unsqueeze(1)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch+1, ' loss: ', total_loss)

AlphaNet(
  (fc1): Linear(in_features=324, out_features=30, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (out): Linear(in_features=30, out_features=1, bias=True)
)


  3%|██▊                                                                                | 1/30 [00:01<00:55,  1.93s/it]

Epoch:  1  loss:  0.04480781889691645


  7%|█████▌                                                                             | 2/30 [00:03<00:53,  1.93s/it]

Epoch:  2  loss:  0.03742689268836047


...

Epoch:  28  loss:  0.01087931409103904


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:56<00:02,  2.06s/it]

Epoch:  29  loss:  0.011289715916854058


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:59<00:00,  1.99s/it]

Epoch:  30  loss:  0.010000619405947824


In [96]:
pred_list = []
label_list = []

for _, (data, label) in enumerate(test_loader):
    data = Variable(data).float()
    pred = alphanet(data)
    pred_list.extend(pred.tolist())
    label_list.extend(label.tolist())

In [97]:
for i in range(1000):  
    print("pred: ", pred_list[i][0] * 100)
    print("true: ",label_list[i][0] * 100)
    print("---------------------")

pred:  0.5327600985765457
true:  0.30959752321982137
---------------------
pred:  -0.037280842661857605
true:  -2.5357142857142887
---------------------
pred:  -2.626555785536766
true:  -0.10672358591248439
---------------------
pred:  2.8465665876865387
true:  0.08643042350907336
---------------------
pred:  -5.135556682944298
true:  -7.183239108746259
---------------------
pred:  8.830100297927856
true:  -19.866071428571438
---------------------
pred:  45.62867283821106
true:  -3.731778425655959
---------------------
pred:  -8.849596977233887
true:  -4.166666666666659
---------------------
pred:  6.436195224523544
true:  -1.746724890829696
---------------------
pred:  -43.47498416900635
true:  -0.4968944099378997
---------------------
pred:  3.315458819270134
true:  0.7978723404255386
---------------------
pred:  9.32169109582901
true:  6.249999999999989
---------------------
pred:  12.153063714504242
true:  -2.5054466230936736
---------------------
pred:  4.932954162359238
true:  1.

true:  0.0
---------------------
pred:  7.396485656499863
true:  5.2525760241266735
---------------------
pred:  -2.0268727093935013
true:  -5.027173913043479
---------------------
pred:  2.0072635263204575
true:  3.9121114683815668
---------------------
pred:  -3.6244507879018784
true:  -3.4482758620689538
---------------------
pred:  2.77910940349102
true:  -1.373626373626381
---------------------
pred:  -3.1124379485845566
true:  1.481274455002798
---------------------
pred:  6.3485607504844666
true:  -4.799513973268535
---------------------
pred:  1.1807393282651901
true:  -0.7128309572301454
---------------------
pred:  -1.4348465949296951
true:  -3.7831021437578793
---------------------
pred:  -4.703139886260033
true:  -1.7241379310344769
---------------------
pred:  5.646625906229019
true:  1.0590015128593
---------------------
pred:  -34.85323786735535
true:  -6.26630269942371
---------------------
pred:  4.047973453998566
true:  -1.6746411483253443
---------------------
pred: 